## **Packages Installation**

In [ ]:
!nvidia-smi

In [ ]:
!pip install tldextract
!pip install sentence_transformers

In [ ]:
!pip install langchain==0.0.228
!pip install chromadb==0.3.26
!pip install sentence_transformer
!pip install auto-gptq==0.2.2
!pip install einops==0.6.1
!pip install unstructured==0.8.0
!pip install transformers
!pip install torch==2.0.1

In [ ]:
from pathlib import Path
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline, StoppingCriteriaList,StoppingCriteria
from langchain.document_loaders import UnstructuredURLLoader, OnlinePDFLoader, PyPDFLoader


## **Creating Data**

In [ ]:
questions_dir = Path("exclusible")
questions_dir.mkdir(exist_ok=True, parents=True)

def write_file(question, answer, file_path):
  text= f"""
  Q: {question}
  A: {answer}
  """.strip()
  with Path(questions_dir / file_path).open("w") as text_file:
    text_file.write(text)


In [ ]:
write_file(
    question="What is Exclusible?",
    answer="""
Exclusible is a premium curated Web3 platform and digital brand that has made a name for itself via its innovative Web3 activations within the luxury arena.

Exclusible’s dynamic offering comprises three main elements: an NFT marketplace for global luxury brands, luxury real estate development within the metaverse, and an elite membership program via an exclusive club – the Alpha NFT Collection.

With these offerings, Exclusible has acquired an extensive global community of luxury collectors. Exclusible operates at the nexus of traditional and digital luxury, leveraging its founders’ experience in hard and soft luxury, cutting-edge technology, cryptocurrency, and marketing to set a new standard in Web3 savoir-faire.
 """.strip(),
    file_path="question_1.txt"
)
write_file(
    question="What is the difference between Exclusible and Opensea (or other NFT marketplaces)?",
    answer="""Exclusible is the go-to platform for luxury in Web 3. These are some of the reasons why we differ from Opensea or other NFT marketplaces:

    Way more than a marketplace: Exclusible is the global brand for luxury in Web3. A destination where brands and collectors alike can connect and engage in new experiences
    The most exclusive membership program: through Exclusible’s collections, we offer our community members a ticket for the most unique IRL and digital experiences and benefits
    Not only for the crypto-experience: our aim is to democratize luxury in Web3, that is why we provide tools to on-board from the most crypto amateurs to the most experienced ones
 """.strip(),
    file_path="question_2.txt"
)
write_file(
    question="How can I attend Exclusible’s parties and IRL events? ",
    answer="""If you want to be part of the Exclusible community and have access to all of the privileges that it provides, you can acquire a NFT from our Alpha or Theia Collections in the secondary market here:

    Exclusible Alpha Collection: https://opensea.io/collection/exclusible-alpha
    Exclusible Theia Collection: https://opensea.io/collection/exclusible-gold-alpha
 """.strip(),
    file_path="question_3.txt"
)
write_file(
    question="What blockchain are Exclusible’s Collections on?",
    answer="""Our platform is built on Ethereum, the biggest blockchain and community for NFTs. Exclusible aims to democratize the access to Web3 for luxury collectors, thus the Ethereum network provides the perfect infrastructure for that to happen. """.strip(),
    file_path="question_4.txt"
)
write_file(
    question="What actions does Exclusible take to address its carbon footprint?",
    answer="""Ethereum is on the way of switching its Proof of Work mechanism for a Proof of Stake one (Ethereum 2.0.). This will represent a reduction of energy consumption by 99,95%. Exclusible plans on adopting Ethereum 2.0. whenever possible. The release is scheduled for mid 2022.""".strip(),
    file_path="question_5.txt"
)

## **Creating LLM Model**

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [ ]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"


tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE)

generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [ ]:
question = (
    "Wich programming language is more suitable for a beginner: Python or Javascript ?"
)

prompt = f"""
### Instruction: {question}
### Response:
""".strip()

print(prompt)

### Instruction: Wich programming language is more suitable for a beginner: Python or Javascript ?
### Response:


In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(DEVICE)

with torch.inference_mode():
  output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

CPU times: user 5.18 s, sys: 215 ms, total: 5.39 s
Wall time: 8.17 s


In [ ]:
print(tokenizer.decode(output[0]))

<s> ### Instruction: Wich programming language is more suitable for a beginner: Python or Javascript ?
### Response:Both Python and JavaScript are great languages for beginners. Python is known for its readability and simplicity, making it easy to learn and write. JavaScript, on the other hand, is a popular language for web development and has a similar syntax to other programming languages. Ultimately, the choice depends on the individual's interests and goals.</s>


*Cleaning :tokenizer.decode(output[0]) to only get the sentence*
👇


In [ ]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)

In [ ]:
from typing import List
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

stop_tokens = [["### Input", ":"], ["### Response", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, DEVICE)]
)

*Creating pipeline to just have to call it later to use LLM*👇

In [ ]:

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
     stopping_criteria=stopping_criteria,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1
)



*** Pipeline:


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

In [ ]:
llm =HuggingFacePipeline(pipeline=pipe)

In [ ]:
response = llm(prompt)

Both Python and JavaScript are great languages for beginners. Python has a simpler syntax, making it easier to learn and read. It also has a strong focus on code readability and cleanliness. On the other hand, JavaScript is widely used in web development and can help you build interactive websites. Ultimately, the choice depends on your interests and goals. If you're interested in data science or backend development, Python may be a better fit. If you want to work with web technologies, JavaScript would be a good option.


## **Chunks and Embeddings**



In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base",
    model_kwargs={"device":DEVICE}
)

## **ChatBot**

In [ ]:
from langchain.schema import BaseOutputParser
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        print("In Cleaning Output")
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\n### Input"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\n### Response"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"


In [ ]:
from langchain.schema import BaseOutputParser
DEFAULT_TEMPLATE= """
### Instruction: Youre name is Albert, you're a customer support agent from a Start up called Exclusible, that is talking to a customer. Use only the chat history and the information in this parenthesis (
The CEO of Exclusible is Thibault Launay.
Every brand name after the url https://www.exclusible.com/, like this one: https://www.exclusible.com/amedeo are projects created by Exclusible.
Exclusible has work with buggati, alpine, hugo boss, ...
 {context}

Your startup sells 4 services:
- NFT creation,
- Metaverse,
- Marketing Agency
- Immersive Real Estate
), to answer in a helpful manner to the question. If you don't know the answer - say contact contact@exclusible.com for more information.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

class ChatBot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        training_data: list,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"], template=prompt_template, output_parser=CleanupOutputParser())
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(training_data, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False
    ):
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            human_prefix="### Input",
            ai_prefix="### Response",
            input_key="question",
            output_key="output_text",
            return_messages=False
        )
        return load_qa_chain(text_pipeline, chain_type="stuff", prompt=prompt, memory=memory, verbose=verbose)

    def _embed_data(self, training_data, embeddings: HuggingFaceEmbeddings) -> Chroma:
        urls = training_data.urls
        pdfs = training_data.pdfs
        online_pdfs = training_data.online_pdfs
        documents_dir = training_data.documents_dir

        documents_loader = []
        if (len(documents_dir) > 0):
            for document_dir in documents_dir:
                documents_loader.append(DirectoryLoader(
                    document_dir, glob="**/*txt"))

        urls_loader = []
        if (len(urls) > 1):
            urls_loader.append(UnstructuredURLLoader(urls=urls))

        pdfs_loader = []
        if (len(pdfs) > 0):
            for pdf in pdfs:
                pdfs_loader.append(PyPDFLoader(pdf))

        online_pdfs_loader = []
        if (len(online_pdfs) > 0):
            for online_pdf in online_pdfs:
                online_pdfs_loader.append(PyPDFLoader(online_pdf))

        loaders = [urls_loader, pdfs_loader,
                   online_pdfs_loader, documents_loader]

        docs = []
        for loader in loaders:
            if (len(loader) > 0):
                docs.extend(loader[0].load())
        #print('docs',docs)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=512, chunk_overlap=40)
        texts = text_splitter.split_documents(docs)
        return Chroma.from_documents(texts, embeddings)

    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})




Get all urls on a **website**

In [ ]:
import requests
from bs4 import BeautifulSoup
import tldextract
import re

class TrainingData():
    def __init__(self):
        self.urls: list
        self.pdfs: list
        self.online_pdfs: list
        self.documents_dir: list


def get_urls(urls):
    all_urls = []
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all('a'):
                href = link.get('href')
                if href != None and len(href) > 0:
                    if '#' not in href and 'javascript' not in href:
                        all_urls.append(href)

            for i in range(len(all_urls)):
                if all_urls[i] != None and len(all_urls[i]) > 0:
                    if '#' not in all_urls[i] and 'javascript' not in all_urls[i]:
                        if all_urls[i] is not None and all_urls[i][0] == '/':
                            all_urls[i] = url + all_urls[i]

            # Removes duplicates
    all_urls = list(set(all_urls))
    return all_urls


def get_subdomain(urls):
    all_subdomains = []
    for url in urls:
        if url != None:
            if '.exclusible' in url:
                if 'https://' in url:
                    subdomain = re.findall(r'https:\/\/(.*?)\.exclusible', url)
                    if subdomain:
                        if subdomain[0] != 'www':
                            if subdomain[0] != 'next':
                                all_subdomains.append(url)
    all_subdomains = list(set(all_subdomains))
    return all_subdomains


url = ['https://exclusible.com']


def get_all_urls(url):
    urls = get_urls(url)
    subdomains = get_subdomain(urls)
    subdomains_urls = get_urls(subdomains)
    all_urls = urls + subdomains_urls
    return all_urls

training_data = TrainingData()
training_data.urls = get_all_urls(url)
training_data.pdfs = []
training_data.online_pdfs = []
training_data.documents_dir = ["./exclusible"]



In [ ]:
# !pip install pandas
# import pandas as pd
# df = pd.DataFrame(training_data.urls, columns=["URLs"])
# output_file = "output.xlsx"
# df.to_excel(output_file, index=False)
# print(training_data.urls)
# print(len(training_data.urls))

In [ ]:
chatbot = ChatBot(llm, embeddings, training_data)

# Loop

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

while True:
  user_input = input("You: ")
  if user_input.lower() in ["bye","stop"]:
    break
  answer = chatbot(user_input)
  print()


You: What is highloop?
   Highloop is another service offered by Exclusible that focuses on providing cutting-edge software development solutions to businesses worldwide. With expertise in blockchain technology, artificial intelligence, and machine learning, Highloop helps companies transform their operations and achieve greater efficiency through customized software applications. Is there anything else I can assist you with or any other questions you would like to know about our services or company culture?

You: bye
